In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from dotenv import load_dotenv
#from sentence_transformers import SentenceTransformer
#import chromadb

In [3]:
url = "https://es.wikipedia.org/wiki/Geograf%C3%ADa_de_Colombia"

# Fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
# Buscar todas las tablas en la página
tables = soup.find_all('table')

# Verificar si alguna tabla contiene información sobre departamentos y capitales
tabla = None
for table in tables:
    if "Departamento" in table.get_text() and "Capital" in table.get_text():
        tabla = table
        break

# Si se encuentra la tabla, convertirla en un DataFrame
if tabla:
    df_departamentos = pd.read_html(str(tabla))[0]
    df_departamentos
else:
    print("No se encontró una tabla con información de departamentos y capitales.")

/tmp/ipykernel_635/765528878.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_departamentos = pd.read_html(str(tabla))[0]


In [4]:
# Eliminar la columna 0 y la primera fila
df_departamentos = df_departamentos.iloc[1:, 1:]

In [5]:
new_column_names = df_departamentos.iloc[0]
df_departamentos.columns = new_column_names.values
df_departamentos = df_departamentos.iloc[1:]

In [6]:
modelo_nombre = 'all-MiniLM-L6-v2'
try:
    model = SentenceTransformer(modelo_nombre)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error: {e}")

Modelo cargado exitosamente.


In [7]:
documentos = []
for index, row in df_departamentos.iterrows():
    documento = f"Departamento: {row['Departamento']}, Capital: {row['Capital']}"
    documentos.append(documento)

In [8]:
try:
    embeddings = model.encode(documentos)
    print(f"Se generaron {len(embeddings)} embeddings.")
    print(f"Dimensión de cada embedding: {embeddings.shape[1]}")
except Exception as e:
    print(f"Error al generar los embeddings: {e}")

Se generaron 35 embeddings.
Dimensión de cada embedding: 384


In [11]:
client =  chromadb.PersistentClient(path="./wikipedia_search_db")

# Para una base de datos persistente en disco:
# client = chromadb.PersistentClient(path="./mi_base_de_datos")

# Nombre de la colección donde guardarás tus embeddings
collection_name = "departamentos_capitales"

# Obtener o crear la colección
collection = client.get_or_create_collection(name=collection_name)

print(f"Colección '{collection_name}' creada o cargada.")

Colección 'departamentos_capitales' creada o cargada.


In [12]:
ids = [f"doc_{i}" for i in range(len(documentos))]  # Generar IDs únicos para cada documento

collection.add(
    embeddings=embeddings,
    documents=documentos,
    ids=ids
)

print(f"Se han añadido {collection.count()} documentos a la colección '{collection_name}'.")

Se han añadido 35 documentos a la colección 'departamentos_capitales'.


## Pregunta del usuario

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb

modelo_nombre = 'paraphrase-MiniLM-L6-v2'
try:
    model = SentenceTransformer(modelo_nombre)
    print(f"Modelo '{modelo_nombre}' cargado exitosamente para la consulta.")
except Exception as e:
    print(f"Error al cargar el modelo para la consulta: {e}")
    exit()

c:\Users\Victor\Desktop\Productos\LLM-Wikipedia-search-region\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Victor\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Modelo 'paraphrase-MiniLM-L6-v2' cargado exitosamente para la consulta.


In [15]:
client = chromadb.PersistentClient(path="./wikipedia_search_db")
collection_name = "departamentos_capitales"
try:
    collection = client.get_collection(name=collection_name)
    print(f"Conectado exitosamente a la colección '{collection_name}'.")
except Exception as e:
    print(f"Error al conectar a la colección '{collection_name}': {e}")
    exit()

Conectado exitosamente a la colección 'departamentos_capitales'.


In [ ]:
def recuperar_documentos_relevantes(consulta, coleccion, modelo_embeddings, num_resultados=1):
    """
    Recupera los documentos más relevantes de la base de datos vectorial para una consulta.

    Args:
        consulta (str): La consulta del usuario en texto plano.
        coleccion (chromadb.Collection): La colección de ChromaDB que contiene los embeddings.
        modelo_embeddings (SentenceTransformer): El modelo de Sentence Transformers para generar embeddings de la consulta.
        num_resultados (int): El número de documentos más relevantes a recuperar (por defecto es 3).

    Returns:
        list: Una lista de los documentos (strings) más relevantes, o None si hay un error.
    """
    try:
        # Generar el embedding de la consulta
        consulta_embedding = modelo_embeddings.encode([consulta])

        # Buscar en la base de datos vectorial
        resultados = coleccion.query(
            query_embeddings=consulta_embedding,
            n_results=num_resultados
        )

        # Devolver los documentos recuperados
        documentos_recuperados = resultados['documents'][0]
        return documentos_recuperados

    except Exception as e:
        print(f"Error durante la recuperación de documentos: {e}")
        return None

### Ejemplo

In [17]:
consulta_usuario = "¿Cuál es la capital de Antioquia?"
documentos_relevantes = recuperar_documentos_relevantes(consulta_usuario, collection, model)

In [19]:
if documentos_relevantes:
    print(f"\nDocumentos relevantes para la consulta '{consulta_usuario}':")
    for doc in documentos_relevantes:
        print(f"- {doc}")
else:
    print("No se pudieron recuperar documentos relevantes.")


Documentos relevantes para la consulta '¿Cuál es la capital de Antioquia?':
- Departamento: Antioquia, Capital: Medellín
- Departamento: Huila, Capital: Neiva
- Departamento: Meta, Capital: Villavicencio


In [20]:
def generar_respuesta_simple(consulta, documentos_contexto):
    """
    Genera una respuesta simple extrayendo información de los documentos de contexto.

    Args:
        consulta (str): La consulta del usuario.
        documentos_contexto (list): Una lista de documentos (strings) relevantes.

    Returns:
        str: Una respuesta simple basada en la información encontrada, o un mensaje indicando que no se encontró información clara.
    """
    if not documentos_contexto:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

    respuesta = f"Basándome en la información encontrada:\n"
    for i, doc in enumerate(documentos_contexto):
        respuesta += f"{i+1}. {doc}\n"

    respuesta += f"\nPara obtener una respuesta más precisa y concisa a tu pregunta: '{consulta}', revisa los documentos proporcionados."
    return respuesta


In [21]:
if documentos_relevantes:
    respuesta_simple = generar_respuesta_simple(consulta_usuario, documentos_relevantes)
    print(f"\nRespuesta simple para '{consulta_usuario}':\n{respuesta_simple}")
else:
    print("No se encontraron documentos relevantes para generar una respuesta.")


Respuesta simple para '¿Cuál es la capital de Antioquia?':
Basándome en la información encontrada:
1. Departamento: Antioquia, Capital: Medellín
2. Departamento: Huila, Capital: Neiva
3. Departamento: Meta, Capital: Villavicencio

Para obtener una respuesta más precisa y concisa a tu pregunta: '¿Cuál es la capital de Antioquia?', revisa los documentos proporcionados.
